In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
import os
import json
import pandas as pd
from selenium.webdriver.support import expected_conditions

In [2]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

#This configuration creates a "browser" object and opens it in "headless" mode 
#(it runs but is not visible, you can see it in "running processes")
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--window-size=1825x1444")
browser = webdriver.Chrome(options = chrome_options)


In [3]:
url_base ='https://www.lazada.sg/'

In [4]:
#request 
browser.get(url_base)

In [5]:
#find element to click
browser.find_element_by_link_text('Babies & Toys').click()

In [6]:

list_subs = browser.find_elements_by_class_name('lzd-site-menu-sub-item')

In [7]:
#Loop to got it only links (Babies & Toys)
full_links = [[i.text ,i.find_element_by_tag_name('a').get_attribute('href')] for i in list_subs if i.text != '']         

In [8]:
#list to get all links
all_links = []

for link in full_links:
    print(f'request : {link[0]}')
    
    browser.get(link[1])
    #Total pages of pagination
    MAX_PAGE = browser.find_element_by_class_name('ant-pagination').find_elements_by_tag_name('li')[-2].text
    MAX_PAGE = int(MAX_PAGE)
    for i in range(MAX_PAGE):

        # The necessary prefix is added so as not to waste time
        all_links.append([link[0],browser.current_url+f'?page={i+1}'])
    
    

request : Mother & Baby
request : Diapers & Wipes
request : Milk Formula & Baby Food
request : Nursing & Feeding
request : Baby Fashion & Accessories
request : Baby Gear
request : Bath & Baby Care
request : Maternity & Nursery
request : Toys & Games
request : Electronic & Remote Control Toys
request : Sports Toys & Outdoor Play
request : Baby & Toddler Toys


In [9]:
print(f'quantity of links {len(all_links)}')

quantity of links 1182


In [10]:
all_links[0]

['Mother & Baby', 'https://www.lazada.sg/mother-baby/?page=1']

In [11]:
#list of products
list_products = []
skips=[]
for i in all_links:
    
    browser.get(i[1])
    #Quantity products
    prods = browser.find_elements_by_class_name('c5TXIP')
    #get each href for product
    for index_prods, prod in enumerate(prods):
        
        #Click in product
        browser.find_elements_by_class_name('c5TXIP')[index_prods].click()
        
        #parse to beautifulsoup
        soup = BeautifulSoup(browser.page_source)
        #get the json 
        metadatos = json.loads(soup.find('script', type='application/ld+json').text)
        #If you can't find it, you didn't click
        try:
            metadatos['sku']
        except:
            #If they fail, we save them and then if you want you can try to scrape them again
            skips.append([index_prods,browser.current_url])
            continue
        
        list_products.append([metadatos['sku'],
                              metadatos['category'],
                              metadatos['name'],
                              metadatos['description'],
                              metadatos['offers']['url'],
                              metadatos['offers']['price']
                             ])
        #Return de page prev.
        browser.execute_script("history.back()")


RuntimeError: No active exception to reraise

In [17]:
browser.quit()

In [ ]:
#kill process. free mem
browser.quit()

In [109]:
list_products[0]

['344274860_MY-977654577',
 'Mother & Baby > Maternity Care > Maternity Wear > Maternity Intimates',
 '(READY STOCK Malaysia) Fello Plus Size Soft Nursing Maternity Bras Women Bra-Cnb Breastfeeding Nursing Pregnant Motherhood Lingerie Wireless Stretchy Bras',
 'https://www.lazada.com.my/products/ready-stock-malaysia-fello-plus-size-soft-nursing-maternity-bras-women-bra-cnb-breastfeeding-nursing-pregnant-motherhood-lingerie-wireless-stretchy-bras-i344274860-s977654577.html',
 14.33]

In [112]:
#parse to dataframe
df = pd.DataFrame(list_products)

In [115]:
#renaming the columns
df.rename(columns={0:'sku',1:'category',2:'name',3:'url',4:'price'},inplace=True)

In [117]:
df.head()

,sku,category,name,url,price
0,344274860_MY-977654577,Mother & Baby > Maternity Care > Maternity Wea...,(READY STOCK Malaysia) Fello Plus Size Soft Nu...,https://www.lazada.com.my/products/ready-stock...,14.33
1,356072211_MY-582833847,Mother & Baby > Maternity Care > Maternity Wea...,(READY STOCK MALAYSIA)(3 Pcs )Lady Soft Nursin...,https://www.lazada.com.my/products/ready-stock...,40.50
2,357908492_MY-506792016,Mother & Baby > Maternity Care > Maternity Wea...,(Ready Stock Malaysia)(3 PCS )Lady Soft Nursin...,https://www.lazada.com.my/products/ready-stock...,39.77
3,463466122_MY-742236082,Mother & Baby > Maternity Care > Maternity Wea...,Mother Breastfeeding Bra Women Maternity Nursi...,https://www.lazada.com.my/products/mother-brea...,4.90
4,441480884_MY-766690713,Mother & Baby > Nursery > Mattresses & Bedding...,[READYSTOK] Muslin swaddle blanket baby newbor...,https://www.lazada.com.my/products/readystok-m...,6.70


In [ ]:
#export to excel. (you can specify the directory)
df.to_excel('test.xlsx')